In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import time
import os

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# base_url = 'https://en.zalando.de'
def get_url(url=''):
    if base_url not in url:
        url = f"{base_url}{url}"
    return BeautifulSoup(requests.get(
        url,
        verify=False,
        timeout=5,
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'en-US,en;q=0.9',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        }
    ).content, 'html.parser')

In [ ]:
home = get_url('/women-home/')

In [52]:
soup = BeautifulSoup(home.content, 'html.parser')

In [28]:
main_links = {}

for li in soup.find_all('nav')[2].find_all('li'):
    main_links[li.get('aria-label')] = li.a.get('href')

In [29]:
main_links

{'Get the Look': '/get-the-look-women/',
 'Clothing': '/clothing/',
 'Shoes': '/shoes/',
 'Sports': '/sports-range/',
 'Accessories': '/accessories/',
 'Beauty': '/beauty-women/?product_group=beauty',
 'Designer': 'https://en.zalando.de/premium-home-women/',
 'Brands': 'https://en.zalando.de/brands/',
 'Sale %': '/outlet/',
 'Pre-owned': '/pre-owned-w/'}

In [30]:
cats = ['Clothing', 'Shoes', 'Sports', 'Accessories']

In [31]:
c = cats[0]

# HnM

In [35]:
base_url = 'https://www2.hm.com'

Getting all top level categories for Damen

In [284]:
main = get_url('/de_de/damen.html')

D:\miniconda3\envs\cs25\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.hm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [285]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []

main_cats = 0

for li in main.find_all('nav')[2].find_all('ul')[0].find_all('ul')[3].find_all('li'):
    li_name = li.a.get_text()[4:].strip()
    if 'Alles zeigen' in li_name:
        continue
    names.append(li_name)
    urls.append(li.a.get('href'))
    parents.append(None)
#     cat_links[li_name] = li.a.get('href')
    main_cats += 1

Getting sub-categories

In [308]:
# lists to store all filter names and values
# filters are extracted at parent category level
filter_names = []
filter_options = []
filter_parents = []

get_filter_options = lambda x: (x.find('span', attrs={'class': 'text'}).text, f"{x.find('input').get('data-name')}={x.find('input').get('value')}")

for i in range(main_cats):
    cat_html = get_url(urls[i])
    
    for li in cat_html.find_all('nav')[2].find_all('ul')[4].find_all('li'):
        if li.a.get('href') == urls[i]:
            for sub_li in li.ul.find_all('li'):
                li_name = sub_li.a.get_text()[4:].strip()
                if 'Alles zeigen' in li_name:
                    continue
                names.append(li_name)
                urls.append(sub_li.a.get('href'))
                parents.append(i)
                
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class':'filter-levels'})[0].ul.contents

    for f in filters_:
        if f != '\n':
            options = list(map(get_filter_options, f.ul.find_all('li')))
            filter_options.extend(options)
            filter_names.extend([f.find(attrs={'class':'filter-name'}).text.strip()]*len(options))
            filter_parents.extend([i]*len(options))


Creating dataframe of categories and sub-categories

In [309]:
cats = pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents
})

In [310]:
cats.head()

,Category,URL,Parent_Category
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN
1,Basics,/de_de/damen/produkte/basics.html,NaN
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN


In [311]:
cats.shape

(281, 3)

In [322]:
cats[cats['Parent_Category'].isna()]['Category'].unique()

array(['Shirts & Tops', 'Basics', 'Hemden & Blusen',
       'Cardigans & Pullover', 'Strickwaren', 'Blazer', 'Jacken & Mäntel',
       'Hosen', 'Jeans', 'Shorts', 'Röcke', 'Kleider',
       'Jumpsuits & Playsuits', 'Schuhe', 'Accessoires', 'Bademode',
       'Unterwäsche', 'Loungewear', 'Nachtwäsche',
       'Socken & Strumpfhosen', 'Sportbekleidung', 'Umstandsmode',
       'Große Größen', 'Beauty', 'Hundebekleidung und -Accessoires',
       'Hoodies & Sweatshirts', 'Pflegeprodukte', 'Petite-Kleidung',
       'Premium Selection'], dtype=object)

In [323]:
cats_to_keep = [
    'Shirts & Tops', 'Basics', 'Hemden & Blusen', 'Cardigans & Pullover', 'Strickwaren', 'Blazer', 
    'Jacken & Mäntel', 'Hosen', 'Jeans', 'Shorts', 'Röcke', 'Kleider', 'Jumpsuits & Playsuits', 'Schuhe', 
    'Accessoires', 'Bademode', 'Unterwäsche', 'Loungewear', 'Nachtwäsche', 'Socken & Strumpfhosen', 
    'Sportbekleidung', 'Beauty', 'Hoodies & Sweatshirts'
]

In [328]:
cats_t = cats[(cats['Parent_Category'].isna()) & (cats['Category'].isin(cats_to_keep))]

In [331]:
cats_t = cats_t.append(cats[cats['Parent_Category'].isin(cats_t.index)])

In [356]:
cats_t.head(30)

,Category,URL,Parent_Category
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN
1,Basics,/de_de/damen/produkte/basics.html,NaN
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN
5,Blazer,/de_de/damen/produkte/blazer-westen.html,NaN
6,Jacken & Mäntel,/de_de/damen/produkte/jacken-maentel.html,NaN
7,Hosen,/de_de/damen/produkte/hosen-leggings.html,NaN
8,Jeans,/de_de/damen/produkte/jeans.html,NaN
9,Shorts,/de_de/damen/produkte/shorts.html,NaN


In [457]:
# saving categories
cats_t.to_parquet('../data/HnM_Categories.parquet')

Creating dataframe of fitler options per category

In [343]:
filters = pd.DataFrame({
    'Parent_Category': filter_parents,
    'Filter_Name': filter_names,
    'Fitler_Option': filter_options,
})

In [344]:
filters.shape

(3218, 3)

In [345]:
filters_t = filters[filters['Parent_Category'].isin(cats_t[cats_t['Parent_Category'].isna()].index)]

In [346]:
filters_t.head(10)

,Parent_Category,Filter_Name,Fitler_Option
0,0,Sortieren nach,"(Empfohlen, sort=stock)"
1,0,Sortieren nach,"(Neuheiten, sort=newProduct)"
2,0,Sortieren nach,"(Niedrigster Preis, sort=ascPrice)"
3,0,Sortieren nach,"(Höchster Preis, sort=descPrice)"
4,0,Conscious choice,"(Conscious choice, sustainabilities=Conscious+..."
5,0,Marke,"(Fila, brandNames=Fila)"
6,0,Marke,"(H&M, brandNames=H%26M)"
7,0,Marke,"(Minga London, brandNames=Minga+London)"
8,0,Marke,"(Superdry, brandNames=Superdry)"
9,0,Marke,"(aim'n, brandNames=aim%27n)"


In [347]:
filters_t.shape

(2589, 3)

In [348]:
filters_t['Filter_Name'].unique()

array(['Sortieren nach', 'Conscious choice', 'Marke', 'Größe', 'Farbe',
       'MUSTER', 'Produkttyp', 'Style', 'Funktion', 'Ausschnitt',
       'Ärmellänge', 'Länge', 'Ärmelstil', 'Kragen', 'Passform', 'Anlass',
       'Konzept', 'Qualität', 'Material', 'Bundhöhe', 'Schuhstil',
       'Absatzhöhe', 'Accessoirestil', 'Deckkraft', 'Finish', 'Hautton',
       'Hauttyp', 'Duftfamilie', 'Nutzen', 'Problemstelle', 'Form',
       'Hauptinhaltsstoff', 'LSF'], dtype=object)

In [349]:
filters_to_keep = [
    'Farbe','MUSTER', 'Style', 'Funktion', 'Ausschnitt','Ärmellänge','Länge', 'Ärmelstil', 'Kragen', 
    'Passform', 'Anlass', 'Konzept', 'Bundhöhe', 'Schuhstil', 'Absatzhöhe', 'Accessoirestil', 'Deckkraft', 
    'Finish', 'Hautton', 'Hauttyp'
]

In [350]:
filters_t = filters_t[filters_t['Filter_Name'].isin(filters_to_keep)]

In [351]:
filters_t.shape

(1456, 3)

In [454]:
filters_t.head()

,Parent_Category,Filter_Name,Fitler_Option
30,0,Farbe,"(Beige, colorWithNames=beige_f5f5dc)"
31,0,Farbe,"(Blau, colorWithNames=blau_0000ff)"
32,0,Farbe,"(Braun, colorWithNames=braun_a52a2a)"
33,0,Farbe,"(Gelb, colorWithNames=gelb_ffff00)"
34,0,Farbe,"(Grau, colorWithNames=grau_808080)"


In [458]:
# saving filters
filters_t.to_parquet('../data/HnM_Filters.parquet')

Iterate through all sub-categories - get all products (images & ids)

In [50]:
# lamda functions for getting various params
img_url = lambda x: x.find_all('img')[0].get('src') if x.find_all('img')[0].get('src') else x.find_all('img')[0].get('data-src')
img_ext = lambda x: re.search('set=source\[[a-z0-9/]*(\.[a-z]*)]', img_url(x))[1]
item_name = lambda x: x.find_all('a')[0].get('title')
item_url = lambda x: x.find_all('a')[0].get('href')
item_code = lambda x: x.get('data-articlecode')

def save_image(code, img, ext):
    if os.path.exists(f"{code}{ext}"):
        return
    img_data = requests.get(img if img[:6] == 'https:' else f"https:{img}")
    target = f"../images/{code}{ext}"
    with open(target, 'wb') as f:
        f.write(img_data.content)

In [ ]:
size = 1000

df_items = pd.DataFrame(columns=['Code', 'Name', 'URL', 'Img_URL', 'Img_Ext', 'Category'])
i = 0
total = cats_t[~cats_t['Parent_Category'].isna()].shape[0]

print(f'Total Categories = {total}')
               
for idx, row in cats_t[~cats_t['Parent_Category'].isna()].iterrows():
    # arrays to hold the details
#     print(f"Fetching {row['URL']}")
    subcat = get_url(f"{row['URL']}?image=stillLife&offset=0&page-size={size}")
    all_items = subcat.find_all('article')
#     print(f"Found {len(all_items)} items.")
    
    codes = list(map(item_code, all_items))
    img_urls = list(map(img_url, all_items))
    img_exts = list(map(img_ext, all_items))
    
    df_items = pd.concat([df_items, pd.DataFrame({
        'Code': codes,
        'Name': list(map(item_name, all_items)),
        'URL': list(map(item_url, all_items)),
        'Img_URL': img_urls,
        'Img_Ext': img_exts,
        'Category': [idx]*len(all_items)
    })], ignore_index=True)
    
    list(map(save_image, codes, img_urls, img_exts))
    
    i += 1
    print(f"Completed {i} - {row['Category']} - Saved {len(all_items)} items.")
    if i%10 == 0:
        time.sleep(1)

Total Categories = 213
Completed 1 - Cut-Out-Tops - Saved 12 items.
Completed 2 - Bralette-Tops - Saved 0 items.
Completed 3 - Bandeau-Tops - Saved 7 items.
Completed 4 - Oberteile mit Puffärmeln - Saved 7 items.


In [ ]:
df_items

In [ ]:
# saving all items
df_items.to_parquet('../data/HnM_Items.parquet')

Iterate through each filter for each sub-category - get all product ids and tag them

In [4]:
cats_t = pd.read_parquet('../data/HnM_Categories.parquet')

In [28]:
filters_t = pd.read_parquet('../data/HnM_Filters.parquet')

In [12]:
cats_t.loc[29].head()

Category                                      Cut-Out-Tops
URL                /de_de/damen/produkte/tops/cut-out.html
Parent_Category                                        0.0
Name: 29, dtype: object

In [29]:
filters_t[filters_t['Parent_Category'] == 0].head()

,Parent_Category,Filter_Name,Filter_Option
30,0,Farbe,"[Beige, colorWithNames=beige_f5f5dc]"
31,0,Farbe,"[Blau, colorWithNames=blau_0000ff]"
32,0,Farbe,"[Braun, colorWithNames=braun_a52a2a]"
33,0,Farbe,"[Gelb, colorWithNames=gelb_ffff00]"
34,0,Farbe,"[Grau, colorWithNames=grau_808080]"


In [48]:
size = 1000

df_attributes = pd.DataFrame(columns=[
    'Code', 'Filter_ID'
])

i = 0
total = filters_t.shape[0]

print(f'Total Filters = {total}')


for idx, row in filters_t:
    subcat_filter = get_url(f"{cats_t.loc[int(row['Parent_Category'])]['URL']}?image=stillLife&offset=0&page-size={size}&{row['Filter_Option'][1]}")
    codes = list(map(item_code, subcat_filter.find_all('article')))
    df_attributes = pd.concat([
        df_attributes,
        pd.DataFrame({
            'Code': codes,
            'Filter_ID': [idx]*len(codes)
        })
    ])
    
    i += 1
    if i%10 == 0:
        print(f"Completed {i}")
        time.sleep(1)

In [51]:
df_attributes.to_parquet('../data/HnM_FilterAttributes.parquet')

['0979236026',
 '1000128008',
 '1000128007',
 '1080623001',
 '1063867001',
 '0572797079',
 '1063196001',
 '1048041008',
 '1052976001',
 '1060970001']